In [1]:
# This script should verify the collected data
# we have collected some stations at certain times during the day
# let's see if the numbers collected match the actual collected data via API

In [5]:
import sqlite3
import pandas as pd

def get_closest_station_and_state(target_lat, target_lon, specific_time):
    # Connect to the SQLite database
    conn = sqlite3.connect('../youbike_data.db')

    # Query to find the closest station
    closest_station_query = """
    WITH DistanceToTarget AS (
        SELECT 
            sno, 
            snaen, 
            latitude, 
            longitude, 
            total AS capacity, 
            available_rent_bikes AS s_init, 
            mday,
            (
                6371 * acos(
                    cos(radians(?)) * cos(radians(latitude)) *
                    cos(radians(longitude) - radians(?)) +
                    sin(radians(?)) * sin(radians(latitude))
                )
            ) AS distance
        FROM youbike_data
    )
    SELECT *
    FROM DistanceToTarget
    ORDER BY distance ASC
    LIMIT 1
    """

    # Execute the query to find the closest station
    closest_station = pd.read_sql_query(
        closest_station_query, 
        conn, 
        params=(target_lat, target_lon, target_lat)
    )

    if closest_station.empty:
        print("No stations found.")
        conn.close()
        return None

    # Get the station number of the closest station
    sno = closest_station.iloc[0]['sno']

    # Query to get the state of the closest station at the specific time
    station_state_query = """
    SELECT sno, snaen, latitude, longitude, total AS capacity, available_rent_bikes AS s_init, mday
    FROM youbike_data
    WHERE sno = ? AND mday = ?
    """

    # Execute the query to get the station state
    station_state = pd.read_sql_query(station_state_query, conn, params=(sno, specific_time))

    # Close the connection
    conn.close()

    return station_state

# Example usage
target_lat = 25.0330  # Replace with your latitude
target_lon = 121.5654  # Replace with your longitude
specific_time = "2024-11-01 11:30:00"  # Replace with your time and date

result = get_closest_station_and_state(target_lat, target_lon, specific_time)
print(result)


Empty DataFrame
Columns: [sno, snaen, latitude, longitude, capacity, s_init, mday]
Index: []


In [1]:
# with sql tool, we can use this query:
    # WITH ClosestStation AS (
    #     SELECT 
    #         sno, 
    #         snaen, 
    #         latitude, 
    #         longitude, 
    #         total AS capacity, 
    #         available_rent_bikes AS s_init, 
    #         mday,
    #         (
    #             6371 * acos(
    #                 cos(radians(?)) * cos(radians(latitude)) *
    #                 cos(radians(longitude) - radians(?)) +
    #                 sin(radians(?)) * sin(radians(latitude))
    #             )
    #         ) AS distance
    #     FROM youbike_data
    #     ORDER BY distance ASC
    #     LIMIT 1
    # )
    # SELECT *
    # FROM youbike_data
    # WHERE sno = (SELECT sno FROM ClosestStation)

In [6]:
# case 1
# 8:58 am 21th November 2024,, close to home (24.999808, 121.547607)
# sno is xingfeng park, 500105024
# 8 bikes available on pic
# data base entry says 5
specific_time = "2024-11-21 08:58:00"  # Replace with your time and date
result = get_closest_station_and_state(24.999808, 121.547607, specific_time)
print(result)

Empty DataFrame
Columns: [sno, snaen, latitude, longitude, capacity, s_init, mday]
Index: []


In [3]:
# case 2
# 3:20 am 22nd November 2024, close to home (24.998934, 121.548893)
# 12 bikes available
# 14 bikes in the db. So I think right now we only record if there is a change in data
# i also think that some of these bikes at the station are not shown or shown as broken